# **Broadleaf weed classification by using Resnet Model**

**IMPORTANT**: Use GPU runtime environment.If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE" 

## 1. Import dependencies

In [ ]:
!pip install import-ipynb
!pip install wandb

In [ ]:
from google.colab import drive
from torchvision import datasets, models, transforms, utils
from os import path
from copy import deepcopy
import torch
import import_ipynb
import torch.nn as nn
import torch.optim as optim

## 2. Install Google drive and set path to Train - Val dataset

In [ ]:
drive.mount('/content/drive')
data_dir = '/content/drive/My Drive/DeepLearning Malezas/familias latifoliadas1/dataset'

In [ ]:
#%cd "/content/drive/My Drive/Colab Notebooks/Codes/"
%cd "/content/drive/My Drive/DeepLearning Malezas/Codes"
import myFunctions

## 3. Data Reading

In [ ]:
images = myFunctions.myImages('/content/drive/My Drive/DeepLearning Malezas/familias latifoliadas 2/DK+CL_full', 224, test_dir = '/content/drive/My Drive/DeepLearning Malezas/familias latifoliadas 2/DK+CL_full')

## 4. Check Device

In [ ]:
images.device

## 5. Check dataset size

In [ ]:
image_datasets = images.image_datasets
dataloaders = images.dataloaders
dataset_sizes = images.dataset_sizes
class_names = images.class_names

In [ ]:
images.imshow()

In [ ]:
dataset_sizes

## 6. Check class names (weed families on dataset) and random sample preview

In [ ]:
class_names

## 7. Download pretrained model version from pytorch and set hyperparameters

(Model Pretrained on Imagenet dataset)

In [ ]:
model = models.resnet50(pretrained=True)

In [ ]:
for param in model.parameters():
  param.requires_grad = False
model.fc = nn.Linear(2048, 4)

In [ ]:
modelGPU = model.to(images.device)
opt = optim.SGD(modelGPU.parameters(), lr=0.001, momentum=0.9)
sch = optim.lr_scheduler.StepLR(opt, step_size=7, gamma=0.1)

## 8. Train weed clasification model

- considering : train-val dataset, training setup and the pretrained model version

In [ ]:
images.train_model(modelGPU, nn.CrossEntropyLoss(), opt, sch, num_epochs=15, name='feature-extraction_traivalDK--->DK')

In [ ]:
torch.save(images.model.state_dict(), "/content/drive/MyDrive/DeepLearning Malezas/Modelos/Resnet50/Resnet_CLDK->CLDK_featurex_.h5")
featex = deepcopy(images.model)

## 8. Train weed clasification model

- considering : train-val dataset, training setup and the pretrained model version

In [ ]:
model = models.resnet50(pretrained=True)

In [ ]:
for param in model.parameters():
  param.requires_grad = True
model.fc = nn.Linear(2048, 4)

In [ ]:
modelGPU = model.to(images.device)
opt = optim.SGD(modelGPU.parameters(), lr=0.001, momentum=0.9)
sch = optim.lr_scheduler.StepLR(opt, step_size=7, gamma=0.1)

In [ ]:
images.train_model(modelGPU, nn.CrossEntropyLoss(), opt, sch, num_epochs=15, name='fine-tunning_TrValDK+CL--->TestDK')

In [ ]:
torch.save(images.model.state_dict(), "/content/drive/MyDrive/DeepLearning Malezas/Modelos/Resnet50/Resnet_CLDK->CLDK_finetune.h5")
finetune = deepcopy(images.model)